In [22]:
import pandas as pd
import csv

In [37]:
with open('./data/inst/knap_4.inst.dat', 'r') as inst_file:
    reader = csv.reader(inst_file, delimiter=' ', skipinitialspace=True)
    
    for row in reader:
        n = int(row[1])
        M = int(row[2])
        
        weights = []
        costs = []
        for i in range(0, n*2 - 1, 2):
            weights.append(int(row[i + 3]))
            costs.append(int(row[i + 4]))
            
#         print('Row:', row[0])
#         print(weights)
#         print(costs)

        # TODO: algorithms
            


Row: 9000
[18, 42, 88, 3]
[114, 136, 192, 223]
Row: 9001
[55, 81, 14, 52]
[29, 64, 104, 222]
Row: 9002
[89, 18, 57, 69]
[196, 62, 34, 112]
Row: 9003
[34, 23, 62, 2]
[169, 152, 44, 224]
Row: 9004
[67, 74, 87, 65]
[145, 111, 139, 243]
Row: 9005
[12, 52, 16, 3]
[66, 167, 150, 180]
Row: 9006
[85, 13, 4, 83]
[190, 165, 47, 198]
Row: 9007
[31, 38, 50, 22]
[96, 77, 1, 202]
Row: 9008
[88, 26, 76, 80]
[235, 224, 220, 194]
Row: 9009
[27, 42, 87, 69]
[163, 22, 179, 61]
Row: 9010
[6, 33, 5, 67]
[36, 253, 1, 44]
Row: 9011
[65, 58, 53, 20]
[241, 150, 26, 218]
Row: 9012
[51, 5, 19, 81]
[87, 198, 40, 47]
Row: 9013
[11, 24, 50, 21]
[166, 165, 74, 100]
Row: 9014
[65, 20, 13, 39]
[38, 243, 107, 122]
Row: 9015
[73, 33, 32, 82]
[107, 213, 17, 116]
Row: 9016
[7, 30, 82, 13]
[193, 95, 57, 164]
Row: 9017
[33, 54, 83, 75]
[165, 141, 132, 54]
Row: 9018
[88, 27, 36, 29]
[218, 157, 97, 229]
Row: 9019
[43, 63, 75, 88]
[103, 227, 87, 152]
Row: 9020
[87, 39, 74, 58]
[229, 57, 34, 139]
Row: 9021
[7, 15, 6, 68]
[196, 